Task 2 of The Forage Quantitative Research Module | JPMC

In [1]:
#imports
import datetime as dt
import pandas as pd
import numpy as np

print("imports complete")

imports complete


In [ ]:


injection_dates = ["2023-6-01", "2023-7-01", "2023-8-01", "2023-9-01"]
withdrawal_dates = ["2023-10-31", "2023-11-30"]

inject_price = 2    #in usd / MMBtu
withdraw_price = 3  # in usd / MMBtu

injec_capacity_rate = 1000000.00  # MMBtu per month
withd_capacity_rate = 1000000.00  # MMBtu per month

maximum_volume = 15000000.00  # MMBtu
stor_cost =  100000.00 #fixed fee per month

injc_withd_cost = 10000.00 # usd per 1 MM MMBtu injected/withdrawn period
transportation = 100000.00 # udd per injection/withdrawal period
def injection_costs():
    print("")
    injection_date = input("Enter injection start date (YYYY-MM-DD): ")
    inject_price = float(input("Enter injection price (usd/MMBtu): "))
    injec_capacity = float(input("Enter injection capacity (MMBtu): "))
    maximum_volume = float(input("Enter maximum storage volume (MMBtu): "))
    stor_cost = float(input("Enter storage cost (usd per 1 million MMBtu): "))

def withdrawal_costs():
    print("")
    withdrawal_date = input("Enter withdrawal start date (YYYY-MM-DD): ")
    withdraw_price = float(input("Enter withdrawal price (usd/MMBtu): "))
    withd_capacity_rate = float(input("Enter withdrawal capacity (MMBtu): "))

injection_costs()
withdrawal_costs()

ValueError: could not convert string to float: ''

In [ ]:
months = 0

for i in range(len(injection_dates)):
    inj_date = dt.datetime.strptime(injection_dates[i], "%Y-%m-%d")
    withd_date = dt.datetime.strptime(withdrawal_dates[i], "%Y-%m-%d")
    delta = withd_date - inj_date
    months += delta.days // 30

